In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "pass1234"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
#df = pd.DataFrame.from_records(shelter.read({})
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Developer: Seth Reeise')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Center Dashboard'))),
    html.Center(html.B(html.H2('Developer: Seth Reeise'))),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        html.A(
        href="https://www.snhu.edu",
        children=[
            html.Img(
                alt="Link to SNHU",
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style = {
                    'width':'10%',
                    'height':'10%',
                    'textAlign':'center'
                }
                )
        ])
        
    ]),
    html.Div([
        dcc.Dropdown(
        id = 'Drop_down',
        options = [{'label':i, 'value':i} for i in ['No Filter/Reset','Water Rescue', 'Mountain or Wilderness Rescue',
                   'Disaster Rescue or Individual Tracking']],
        value = 'No Filter/Reset',
            multi=False,
        clearable=False

        )
    ], 
        style={'width': '48%', 'display':'inline-block'}
    ),
    
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,    
    ),

    html.Br(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph('pie-chart'),
            className='col s10 m5',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id', 'data'),
    [Input('Drop_down', 'value')])
def update_dashboard(filter):
    
    if (filter == 'Water Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}))
    elif (filter == 'Mountain or Wilderness Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}))
    elif (filter == 'Disaster Rescue or Individual Tracking'):
        df = pd.DataFrame.from_records(shelter.readAll({
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20}, "age_upon_outcome_in_weeks":{"$lte":300}}))
    elif (filter == 'No Filter/Reset'):
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    return df.to_dict('records')
        
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    
    Output('pie-chart', 'figure'),
    [Input('Drop_down', 'value')]
    )

def update_graphs(my_dropdown):
    

    if (my_dropdown == 'Water Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({
                "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome":"Intact Female",
                "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}))
        name = 'breed'
        chart_title = 'Breed'
    elif (my_dropdown == 'Mountain or Wilderness Rescue'):
        df = pd.DataFrame.from_records(shelter.readAll({
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}))
        name = 'breed'
        chart_title = 'Breed'
    elif (my_dropdown == 'Disaster Rescue or Individual Tracking'):
        df = pd.DataFrame.from_records(shelter.readAll({
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20}, "age_upon_outcome_in_weeks":{"$lte":300}}))
        name = 'breed'
        chart_title = 'Breed'
    else :
        df = pd.DataFrame.from_records(shelter.readAll({}))
        name = 'animal_type'
        chart_title = 'Animal type'
    
    dff = df
    piechart = px.pie(
                data_frame=dff,
                names= name,
                title= chart_title,
                hole=.2
                )
    return (piechart)
    


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
# FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    location_lat = round((dff.iloc[0,13]), 2)
    location_long = round((dff.iloc[0,14]), 2)
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[location_lat,location_long], children=[
                # Hover over marker
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    # Click on marker
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app